In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.8 MB 3.2 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 6.5 MB 46.2 MB/s 
     |████████████████████████████████| 895 kB 52.6 MB/s 
     |████████████████████████████████| 596 kB 61.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 3.2 MB/s 
     |████████████████████████████████| 134 kB 61.2 MB/s 
     |████████████████████████████████| 212 kB 64.1 MB/s 
     |████████████████████████████████| 1.1 MB 60.4 MB/s 
     |████████████████████████████████| 127 kB 38.6 MB/s 
     |████████████████████████████████| 271 kB 49.5 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 144 kB 69.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [ ]:
import numpy as np
import pandas as pd
import re
import random
import torch
from tqdm.notebook import tqdm
import transformers
from datasets import load_dataset
import textwrap

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
dataset = load_dataset("huggingartists/scriptonite")

Using custom data configuration default


Generating train split: 0 examples [00:00, ? examples/s]

Dataset lyrics_dataset downloaded and prepared to /root/.cache/huggingface/datasets/huggingartists___lyrics_dataset/default/1.0.0/6e291d5b355c7975b3d4f42897db2aeb8486da5a4e48ba02b59740555a91be78. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

In [ ]:
dataset = ' '.join(dataset['train']['text'])

In [ ]:
dataset = dataset.replace('\n', ' ')

In [ ]:
tokens = tokenizer.encode(dataset, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))

187650


In [ ]:
tokens.shape

(187650,)

In [ ]:
from transformers import GPT2LMHeadModel, AdamW

model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [ ]:
batch_size = 10
max_len = 256
epochs = 20

train = len(tokens)//(batch_size*max_len)

# устанавливаем оптимизатор
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

# трансформеры с трудом обучаются, для них нужны разные способы повышения 
# эффективности градиентного спуска
total_steps = train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
# готовим тензоры для обучения

def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


# обучающий цикл
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    model.train()
    pbar = tqdm(range(train))
    for i in pbar:
        batch_ids = prep_tensors(tokens, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                                token_type_ids=None, 
                                
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()


epoch 1/20 : training


  0%|          | 0/73 [00:00<?, ?it/s]

epoch 2/20 : training


  0%|          | 0/73 [00:00<?, ?it/s]

epoch 3/20 : training


...

  0%|          | 0/73 [00:00<?, ?it/s]

epoch 19/20 : training


  0%|          | 0/73 [00:00<?, ?it/s]

epoch 20/20 : training


  0%|          | 0/73 [00:00<?, ?it/s]

In [ ]:
torch.save(model, '../model_weights/sryptonite.pth')